In [ ]:
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import gurobipy as gp
from gurobipy import Model, quicksum, GRB
import time
import pyepo
from pyepo.model.grb import optGrbModel
from pyepo.data.dataset import optDataset
from pyepo.func import SPOPlus
from pyepo.metric import regret  # Import the regret metric from PyEPO
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Set seeds for reproducibility
def set_seeds(seed=42):
    random.seed(seed)                    # Sets seed for Python's built-in random module
    np.random.seed(seed)                 # Sets seed for NumPy's random number generator
    torch.manual_seed(seed)              # Sets seed for PyTorch's CPU operations
    torch.cuda.manual_seed_all(seed)     # Sets seed for PyTorch's GPU operations
    
    # Make sure to disable CuDNN's non-deterministic optimizations
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# the dataset has hourly data from 1 jan2024 to 24Feb2025 (2025 data will be used for tese in the final run = 54 days). 
test_size = 14 #in days - establishes the train/test split (each day has 24 hours/samples)
walk_forward_steps = 14 #in days - controls the evaluation process length (each day has 24 hours)



####################################################################################
####################################################################################


# Data class for battery parameters
class Data:
    def __init__(self, intervals, max_charge, max_discharge, initial_energy_level, battery_capacity, minimum_SoC):
        self.intervals = intervals
        self.max_charge = max_charge
        self.max_discharge = max_discharge
        self.initial_energy_level = initial_energy_level
        self.battery_capacity = battery_capacity
        self.minimum_SoC = minimum_SoC

############## Optimisation Model ##############
# Battery scheduling optimization model
class BatteryScheduling(optGrbModel):
    def __init__(self, data):
        self.data = data
        super().__init__()

    def _getModel(self):
        m = gp.Model("batteryModel")
        m.setParam("OutputFlag", 0)
        intervals = self.data.intervals

        #BIGM = 99999999  # A large constant used to control constraints logically
        
        #### Decision variables ####
        g = m.addVars(intervals, vtype=GRB.CONTINUOUS, name="g")  # Amount of energy sent from grid to Home at interval t
        b = m.addVars(intervals, vtype=GRB.CONTINUOUS, name="b")  # Amount of Energy sent from Battery (discharge) to Home at interval t
        c = m.addVars(intervals, vtype=GRB.CONTINUOUS, name="c")  # Amount of Energy sent from Grid to Battery (charge) at interval t
        z = m.addVars(intervals, vtype=gp.GRB.BINARY, name="z")  # Charge/discharge switch (Binary flag: battery discharged at interval t)
        energy_level = m.addVars(range(intervals + 1), lb=0, ub=self.data.battery_capacity, name="energy_level_kwh") # Battery energy level at each interval
        
        ##### Constraints ####
        # Ensure total energy supply (grid + battery) meets or exceeds demand at every interval
        # Logical constraints linking continuous variables to their binary flags
        # Logical constraints linking continuous variables to their binary flags
        #m.addConstrs(g[t] <= BIGM * y[t] for t in data.intervals)
        #Ensure the initial battery level at interval 0
        # Battery charge and discharge dynamics across intervals
        # Battery cannot discharge more than its current energy level
        # Battery cannot charge and discharge simultaneously at the same interval
        # Battery physical constraints: max charge/discharge and capacity limits
        #Energy level of the battery can not exceed the battery capacity at any interval.
        # Energy level of the battery can not exceed the battery capacity at any interval.
        self.demand_constrs = m.addConstrs((g[t] + b[t] >= 0 for t in range(intervals)), name="demand")
        m.addConstrs(b[t] <= self.data.max_discharge * z[t] for t in range(intervals))
        m.addConstr(energy_level[0] == self.data.initial_energy_level)
        m.addConstrs(energy_level[t + 1] == energy_level[t] + c[t] - b[t] for t in range(intervals))
        m.addConstrs(b[t] <= energy_level[t] for t in range(intervals))
        m.addConstrs(c[t] <= self.data.max_charge * (1 - z[t]) for t in range(intervals))
        m.addConstrs(energy_level[t] >= self.data.battery_capacity * self.data.minimum_SoC for t in range(intervals + 1))
        
        #### Objective function #### : minimise cost of energy purchased from grid and a minimal penalty for battery usage
        m.setObjective(0, gp.GRB.MINIMIZE)
        self._vars = [g[t] for t in range(intervals)] + [b[t] for t in range(intervals)] + [c[t] for t in range(intervals)]
        return m, self._vars

    def setDemand(self, demand):
        """Set demand constraints using predicted/actual demand values"""
        for t in range(self.data.intervals):
            self.demand_constrs[t].RHS = max(demand[t], 0)

    def setObj(self, cost):
        """Set objective with cost vector for [grid power, battery discharge, battery charge]"""
        m = self._model
        if len(cost) == self.data.intervals:
            # If only price vector is provided, expand it to full cost vector format
            intervals = self.data.intervals
            full_cost = np.concatenate([cost, np.zeros(intervals), cost])
            m.setObjective(gp.quicksum(full_cost[i] * self._vars[i] for i in range(len(self._vars))), gp.GRB.MINIMIZE)
        else:
            # Full cost vector is already provided
            m.setObjective(gp.quicksum(cost[i] * self._vars[i] for i in range(len(self._vars))), gp.GRB.MINIMIZE)


    #### Solve optimisation model ####
    def solve(self):
        """Solve the optimization model and return solution and objective value"""
        self._model.optimize()
        if self._model.status == gp.GRB.OPTIMAL:
            sol = [var.x for var in self._vars]
            obj = self._model.objVal
            return sol, obj
        else:
            # If no optimal solution found, return a default solution
            intervals = self.data.intervals
            default_sol = [max(self.demand_constrs[t].RHS, 0) for t in range(intervals)] + [0] * (2 * intervals)
            default_obj = sum(default_sol[t] * (self._model.getObjective().getVar(t).obj if t < intervals else 0) for t in range(intervals))
            return default_sol, default_obj

####################################################################################
####################################################################################

############## MLP model to predict both price and demand ##############
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim=48):  # Output both price and demand (24+24=48)
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.relu(x)  

# This class wraps our combined model to extract only price predictions for regret calculation (to make it work with PyEPO)
class PriceModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def forward(self, x):
        outputs = self.model(x)
        # Extract only price predictions (first 24 values)
        return outputs[:, :24]

# Prepare data with sliding windows (creates the feature window, using the previous 24 hours of data to predict the next 24 hours)
def prepare_data(df, lookback=24, step=24):
    """
    Create sliding window data from time series with a specified step size
    Returns:
        - x: Input features (lookback window)
        - demand: True demand values for each window
        - price: True price values for each window
        - cost: Full cost vectors
        - timestamps: Timestamps for each window
    """
    x, demand, price, timestamps = [], [], [], []
    for t in range(lookback, len(df) - 24 + 1, step):
        # Extract lookback window for features
        # feature_cols = [col for col in df.columns if col not in ['Demand', 'Price']]
        # x_t = df[feature_cols].iloc[t - lookback:t].values.flatten()
        
        # Include ALL columns for lookback period (including yesterday's Demand and Price)
        feature_cols = df.columns
        x_t = df[df.columns].iloc[t - lookback:t].values.flatten()
        
        # Extract next 24 hours for targets
        demand_t = df['Demand'].iloc[t:t + 24].values #Target - Next 24 hours of Demand
        price_t = df['Price'].iloc[t:t + 24].values   # Target - Next 24 hours of Price
        timestamps_t = df.index[t:t + 24]

        # Double-Check Features/targets
        #print(f"Features from {df.index[t - lookback]} to {df.index[t - 1]}")
        #print(f"Targets from {df.index[t]} to {df.index[t + 23]}")
        #Print details for the first window only to Double-Check Features/targets
        np.set_printoptions(suppress=True, precision=4) #control how numbers are printed
        if t == lookback:
            print(f"Features from {df.index[t - lookback]} to {df.index[t - 1]}")
            print(f"Feature values (first 30): {x_t[:30]}... (total {len(x_t)} values)")
            print(f"Targets (Demand) from {df.index[t]} to {df.index[t + 23]}")
            print(f"Demand target values (first 30): {demand_t[:30]}...")
            print(f"Targets (Price) from {df.index[t]} to {df.index[t + 23]}")
            print(f"Price target values (first 30): {price_t[:30]}...")
        
        x.append(x_t)
        demand.append(demand_t)
        price.append(price_t)
        timestamps.append(timestamps_t)
    
    x = np.array(x, dtype=np.float32)
    demand = np.array(demand, dtype=np.float32)
    price = np.array(price, dtype=np.float32)
    # Create cost vectors by combining price with zeros for battery discharge
    cost = np.concatenate([price, np.zeros_like(price), price], axis=1)
    return x, demand, price, cost, timestamps

# Compute true solutions using actual data
def compute_true_solutions(optmodel, demand_data, cost_data):
    """Calculate optimal solutions using true demand and price values"""
    w_true, z_true = [], []
    for demand_t, cost_t in zip(demand_data, cost_data):
        optmodel.setDemand(demand_t)
        optmodel.setObj(cost_t)
        sol, obj = optmodel.solve()
        w_true.append(sol)
        z_true.append(obj)
    return np.array(w_true), np.array(z_true)

############## Training function with detailed metrics ##############
def train_model(reg, func, method_name, train_loader, train_full_loader, train_data, test_data, 
                optmodel, device, step, epochs=1, window_size=None):
    """
    Train and evaluate a model with detailed metrics tracking
    Args:
        reg: Neural network model
        func: Loss function (varies by method)
        method_name: Name of the method (2-Stage, SPO+, DBB) ---->> based on PyEOP API
        train_loader: DataLoader for PyEPO methods
        train_full_loader: DataLoader for 2-Stage method
        train_data: Tuple of (x_train, demand_train, price_train, cost_train)
        test_data: Tuple of (x_test, demand_test, price_test, cost_test)
        optmodel: Battery scheduling optimisation model
        device: Computing device (CPU/GPU)
        step: Current step in walk-forward evaluation (1 step = 24 hours = 1 day)
        epochs: Number of training epochs
        window_size: Size of training window (each window is 24 hours = 1 day)
    Returns:
        epoch_metrics: Dictionary with detailed metrics for each epoch
        predictions: Price and demand predictions
        final_test_regret: Test regret after training
        total_time: Total training time
    """
    # Unpack test_data to get x_test first
    x_test, demand_test, price_test, cost_test = test_data
    
    print(f"\nTraining {method_name} for step {step+1} of {walk_forward_steps}")
    print(f"Training set: {window_size} samples, Test set: {len(x_test)} samples")
    
    # Initialize metrics tracking
    epoch_metrics = {
        'Method': [],
        'Step': [],
        'Window_Size': [],
        'Epoch': [],
        'Train_Regret': [],
        'Test_Regret': [],
        'Epoch_Time': []
    }
    
    x_train, demand_train, price_train, cost_train = train_data
    x_test, demand_test, price_test, cost_test = test_data

    start_time_total = time.time()
       
    optimizer = torch.optim.Adam(reg.parameters(), lr=1e-5)
    l1 = nn.L1Loss()
    l2 = nn.MSELoss()
    
    # Create dataset for PyEPO's regret calculation
    train_opt_dataset = optDataset(optmodel, x_train, cost_train)
    train_opt_loader = DataLoader(train_opt_dataset, batch_size=32, shuffle=False)
    
    test_opt_dataset = optDataset(optmodel, x_test, cost_test)
    test_opt_loader = DataLoader(test_opt_dataset, batch_size=32, shuffle=False)
    
    # Create a wrapper model that extracts only price predictions for regret calculation
    price_model_wrapper = PriceModelWrapper(reg)
    
    for epoch in range(epochs):
        epoch_start = time.time()
        reg.train()
        epoch_loss = 0
        batch_count = 0
        
        if method_name == "2-Stage":
            # For 2-Stage, we use MSE on combined price and demand predictions
            for x_batch, targets_batch in train_full_loader:
                x_batch, targets_batch = x_batch.to(device), targets_batch.to(device)
                predictions = reg(x_batch)
                loss = func(predictions, targets_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                batch_count += 1
        else:
            # For all other methods, use decision-focused training
            for batch_idx, (x_batch, c_batch, w_batch, z_batch, true_demand_batch) in enumerate(train_loader):
                x_batch, c_batch = x_batch.to(device), c_batch.to(device)
                w_batch, z_batch = w_batch.to(device), z_batch.to(device)
                true_demand_batch = true_demand_batch.to(device) 
                # Get model predictions
                predictions = reg(x_batch)
                price_pred = predictions[:, :24]
                demand_pred = predictions[:, 24:]
                
                # Create cost vector from price predictions
                cost_pred = torch.cat([price_pred, torch.zeros_like(price_pred), price_pred], dim=1)
                
                # Compute loss based on method type
                if method_name == "SPO+":
                    # Need to handle demand predictions separately
                    loss = func(cost_pred, c_batch, w_batch, z_batch)
                elif method_name == "DBB":
                    wp = func(cost_pred)
                    zp = torch.sum(wp * c_batch, dim=1).view(-1, 1)
                    loss = l1(zp, z_batch) # Linear loss (β=1) or L2 Quadratic loss (β=2)
                
                # Add MSE term for demand accuracy - critical since predicted demand serves as constraints in optimisation
                # 0.1: Prioritises decision optimisation over accurate demand prediction
                # 0.5: Equal balance between decision quality and demand accuracy
                # 0.9: Emphasises accurate demand forecasting over optimal decisions
                loss += 0.5 * l2(demand_pred, true_demand_batch)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                batch_count += 1
        
        epoch_end = time.time()
        epoch_time = epoch_end - epoch_start
        
        # Calculate regrets after each epoch using PyEPO's regret function
        # Set model to eval mode for regret calculation
        reg.eval()
        price_model_wrapper.eval()
        
        # Update demand in optimization model before calculating regret
        # We need to manually set demand for each sample before regret calculation
        # since the PyEPO regret function doesn't handle demand constraints
        
        # Hack for training regret: We'll calculate it for each sample and take the average
        train_regret_vals = []
        
        # We need to manually calculate regret for each sample to handle demand
        for i in range(len(x_train)):
            # Create single-sample optimDataset
            sample_opt_dataset = optDataset(optmodel, x_train[i:i+1], cost_train[i:i+1])
            sample_loader = DataLoader(sample_opt_dataset, batch_size=1, shuffle=False)
            
            # Set correct demand for this sample
            optmodel.setDemand(demand_train[i])
            
            # Calculate regret using PyEPO's function
            train_regret_i = pyepo.metric.regret(price_model_wrapper, optmodel, sample_loader)
            train_regret_vals.append(train_regret_i)
        
        train_regret = np.mean(train_regret_vals)
        
        # Same for test regret
        test_regret_vals = []
        test_price_preds = []
        test_demand_preds = []
        
        for i in range(len(x_test)):
            # Make predictions for this sample
            with torch.no_grad():
                outputs = reg(torch.tensor(x_test[i:i+1], dtype=torch.float32).to(device))
                price_pred = outputs[0, :24].cpu().numpy()
                demand_pred = outputs[0, 24:].cpu().numpy()
                test_price_preds.append(price_pred)
                test_demand_preds.append(demand_pred)
            
            # Create single-sample optimDataset
            sample_opt_dataset = optDataset(optmodel, x_test[i:i+1], cost_test[i:i+1])
            sample_loader = DataLoader(sample_opt_dataset, batch_size=1, shuffle=False)
            
            # Set correct demand for this sample
            optmodel.setDemand(demand_test[i])
            
            # Calculate regret using PyEPO's function
            test_regret_i = pyepo.metric.regret(price_model_wrapper, optmodel, sample_loader)
            test_regret_vals.append(test_regret_i)
            
            if epoch == epochs - 1:  # Print details on last epoch
                print(f"Sample {i}: Regret = {test_regret_i:.4f}")
        
        test_regret = np.mean(test_regret_vals)
        
        avg_loss = epoch_loss / max(1, batch_count)
        
        # Store epoch metrics
        epoch_metrics['Method'].append(method_name)
        epoch_metrics['Step'].append(step+1)
        epoch_metrics['Window_Size'].append(window_size)
        epoch_metrics['Epoch'].append(epoch+1)
        epoch_metrics['Train_Regret'].append(train_regret)
        epoch_metrics['Test_Regret'].append(test_regret)
        epoch_metrics['Epoch_Time'].append(epoch_time)
        
        if True:  # Always print every epoch
            print(f"Epoch {epoch+1}: Loss {avg_loss:.4f}, Train regret: {train_regret:.4f}, Test regret: {test_regret:.4f}")
    
    # Final evaluation - same as last epoch
    final_train_regret = train_regret
    final_test_regret = test_regret
    predictions = (np.array(test_price_preds), np.array(test_demand_preds))
    
    total_time = time.time() - start_time_total
    print(f"Method {method_name} - Avg Train Regret: {np.mean(epoch_metrics['Train_Regret']):.4f}, Avg Test Regret: {np.mean(epoch_metrics['Test_Regret']):.4f}, Avg Epoch Time: {np.mean(epoch_metrics['Epoch_Time']):.2f}s")
    
    return epoch_metrics, predictions, final_test_regret, total_time

### RMSE ###
def calculate_forecasting_errors(predictions_df):
    """Calculate RMSE for each method's price and demand predictions"""
    
    # Initialize results dictionary
    metrics = {'Method': []}
    
    # Extract true values
    true_price = predictions_df['True_Price'].values
    true_demand = predictions_df['True_Demand'].values
    
    # Get all method names
    method_names = [col.replace('_Price', '') for col in predictions_df.columns 
                   if col.endswith('_Price') and col != 'True_Price']
    
    # Calculate RMSE for each method
    for method in method_names:
        # Get predictions
        pred_price = predictions_df[f'{method}_Price'].values
        pred_demand = predictions_df[f'{method}_Demand'].values
        
        # Filter out NaN values
        valid_price_idx = ~np.isnan(pred_price)
        valid_demand_idx = ~np.isnan(pred_demand)
        
        # Calculate RMSE using sklearn
        price_rmse = np.sqrt(mean_squared_error(true_price[valid_price_idx], 
                                              pred_price[valid_price_idx])) if any(valid_price_idx) else np.nan
        demand_rmse = np.sqrt(mean_squared_error(true_demand[valid_demand_idx], 
                                               pred_demand[valid_demand_idx])) if any(valid_demand_idx) else np.nan
        
        # Add to results
        metrics['Method'].append(method)
        metrics.setdefault('Price_RMSE', []).append(price_rmse)
        metrics.setdefault('Demand_RMSE', []).append(demand_rmse)
        
    # Create DataFrame and save to CSV
    metrics_df = pd.DataFrame(metrics)
    metrics_df.to_csv('forecasting_errors.csv', index=False)
    
    return metrics_df

class CustomOptDataset(torch.utils.data.Dataset):
    def __init__(self, opt_dataset, true_demand):
        self.opt_dataset = opt_dataset
        self.true_demand = true_demand
    
    def __len__(self):
        return len(self.opt_dataset)
    
    def __getitem__(self, idx):
        x, c, w, z = self.opt_dataset[idx]
        demand = torch.tensor(self.true_demand[idx], dtype=torch.float32)
        return x, c, w, z, demand

def main():

    set_seeds()

    global test_size  # in days
    global walk_forward_steps  # in days
    
    # Load data
    df = pd.read_csv("./Data/Demand_Price_1320_withFE.csv")
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.set_index('Timestamp')
    
    # Initialize optimization model
    optmodel = BatteryScheduling(Data(
        intervals=24, max_charge=5, max_discharge=4.5,
        initial_energy_level=10, battery_capacity=50, minimum_SoC=0.10
    ))
    
    # Prepare data with sliding windows
    lookback = 24
    x, demand, price, cost, timestamps = prepare_data(df, lookback, step=24)
    
    # Use CPU for computation
    device = torch.device("cpu")
    print(f"Using device: {device}")

    num_processes = 0  # 1 for single-core, 0 for all of cores
    
    # Define methods to evaluate - 2-Stage, SPO+, and DBB
    methods_list = [
        ("2-Stage", nn.MSELoss()),
        ("SPO+", pyepo.func.SPOPlus(optmodel, processes=num_processes)),
        ("DBB", pyepo.func.blackboxOpt(optmodel, processes=num_processes))
    ]

    # Print experiment configuration
    print(f"\n=== EXPERIMENT CONFIGURATION ===")
    print(f"Total data available: {len(df)} hours")
    print(f"Test size: {test_size} windows")
    print(f"Walk-forward steps: {walk_forward_steps}")
    print(f"Lookback window: {lookback} hours")
    print(f"Number of input features (24 hours lookback * number of columns/features): {lookback * df.shape[1]}")
    print(f"Methods to evaluate: {len(methods_list)}")
    print(f"==============================\n")    
    
    total_windows = len(x)
    print(f"Total windows: {total_windows}, Test windows: {test_size}")

    
    # Results containers
    detailed_results = []
    method_summary = []
    predictions_data = {}
    
    # Store all epoch metrics
    all_epoch_metrics = {
        'Method': [], 'Step': [], 'Window_Size': [], 'Epoch': [],
         'Train_Regret': [], 'Test_Regret': [], 'Epoch_Time': []
    }
    
    # Store true values for test set
    test_true_values = {
        'Timestamp': [], 'True_Price': [], 'True_Demand': []
    }
    
    # Walk-forward evaluation
    all_methods = methods_list
    
    for method_index, (method_name, loss_func) in enumerate(all_methods):
        print(f"\nEvaluating method: {method_name}")
        method_results = {
            'Method': method_name,
            'Step': [],
            'Window Size': [],
            'Test Regret': [],
            'Processing Time': []
        }
        
        # Initialise dictionary to store predictions
        predictions_data[method_name] = {'Price': [], 'Demand': []}
        
        for step in range(walk_forward_steps):
            # Calculate indices for current walk-forward window
            test_window_idx = total_windows - test_size + step
            if test_window_idx >= total_windows:
                print(f"Step {step+1}: No more test data available.")
                break
                
            # Define train and test sets
            x_train = x[:test_window_idx]
            demand_train = demand[:test_window_idx]
            price_train = price[:test_window_idx]
            cost_train = cost[:test_window_idx]
            
            x_test = x[test_window_idx:test_window_idx+1]
            demand_test = demand[test_window_idx:test_window_idx+1]
            price_test = price[test_window_idx:test_window_idx+1]
            cost_test = cost[test_window_idx:test_window_idx+1]
            timestamps_test = timestamps[test_window_idx:test_window_idx+1]

            # Save the last day's test data for verification
            if step == walk_forward_steps - 1 and method_index == 0:
                pd.DataFrame({'feature_index': range(len(x_test[0])), 'feature_value': x_test[0], 'timestamp': str(timestamps_test[0][0])}).to_csv('last_day_features.csv', index=False)
                pd.DataFrame({'timestamp': timestamps_test[0], 'true_price': price_test[0], 'true_demand': demand_test[0]}).to_csv('last_day_targets.csv', index=False)

            
            # Store true values for test windows (only once for first method)
            if method_index == 0:
                for ts, p, d in zip(timestamps_test[0], price_test[0], demand_test[0]):
                    test_true_values['Timestamp'].append(ts)
                    test_true_values['True_Price'].append(p)
                    test_true_values['True_Demand'].append(d)
            
            # Compute true solutions for dataset preparation
            w_train, z_train = compute_true_solutions(optmodel, demand_train, cost_train)
            
            # Prepare combined price and demand targets for 2-Stage method
            combined_targets = np.concatenate([price_train, demand_train], axis=1)
            
            # Prepare datasets - standard PyEPO dataset for decision-focused methods
            opt_dataset = optDataset(optmodel, x_train, cost_train)
            
            # Create a custom dataset that includes true demand values
            train_dataset = CustomOptDataset(opt_dataset, demand_train)
            train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
            
            # Custom dataset for 2-Stage that includes both price and demand targets
            train_full_loader = DataLoader(TensorDataset(
                torch.tensor(x_train, dtype=torch.float32), 
                torch.tensor(combined_targets, dtype=torch.float32)
            ), batch_size=32)
            
            # Initialize model - output dimension  48 (24 price + 24 demand)
            input_dim = lookback * df.shape[1]
            reg = MLP(input_dim=input_dim, hidden_dim1=256, hidden_dim2=128, output_dim=48).to(device)
            
            # Train and evaluate with detailed metrics
            train_data = (x_train, demand_train, price_train, cost_train)
            test_data = (x_test, demand_test, price_test, cost_test)
            try:
                window_size = len(x_train)
                epoch_metrics, predictions, test_regret, proc_time = train_model(
                    reg, loss_func, method_name, 
                    train_loader, train_full_loader, 
                    train_data, test_data, optmodel, device, step,
                    epochs=1, window_size=window_size
                )
                
                # Store detailed metrics for all epochs
                for key in all_epoch_metrics:
                    all_epoch_metrics[key].extend(epoch_metrics[key])
                
                # Store summary for this step
                method_results['Step'].append(step+1)
                method_results['Window Size'].append(window_size)
                method_results['Test Regret'].append(test_regret)
                method_results['Processing Time'].append(proc_time)
                
                # Store predictions - includes both price and demand
                price_pred, demand_pred = predictions
                predictions_data[method_name]['Price'].extend(price_pred[0])
                predictions_data[method_name]['Demand'].extend(demand_pred[0])
                
            except Exception as e:
                print(f"Error training {method_name} at step {step+1}: {e}")
                method_results['Step'].append(step+1)
                method_results['Window Size'].append(len(x_train))
                method_results['Test Regret'].append(float('nan'))
                method_results['Processing Time'].append(float('nan'))
                
                # Fill with NaN for predictions
                predictions_data[method_name]['Price'].extend([float('nan')] * 24)
                predictions_data[method_name]['Demand'].extend([float('nan')] * 24)
        
        # Add the method's results to the overall summary
        for i in range(len(method_results['Step'])):
            detailed_results.append({
                'Method': method_name,
                'Step': method_results['Step'][i],
                'Window Size': method_results['Window Size'][i],
                'Test Regret': method_results['Test Regret'][i],
                'Processing Time (s)': method_results['Processing Time'][i]
            })
        
        # Compute method summary
        avg_test_regret = np.nanmean(method_results['Test Regret'])
        avg_processing_time = np.nanmean(method_results['Processing Time'])
        total_processing_time = np.nansum(method_results['Processing Time'])
        
        method_summary.append({
            'Method': method_name,
            'Avg Test Regret': avg_test_regret,
            'Avg Processing Time (s)': avg_processing_time,
            'Total Processing Time (s)': total_processing_time,
            'Steps Completed': len(method_results['Step'])
        })
    
    # Create DataFrames for different result types
    df_epoch_metrics = pd.DataFrame(all_epoch_metrics)
    df_detailed = pd.DataFrame(detailed_results)
    df_summary = pd.DataFrame(method_summary)
    
    # Create predictions DataFrame with true values
    df_true_values = pd.DataFrame(test_true_values)
    
    # Add method predictions for both price and demand
    for method_name in predictions_data:
        if len(predictions_data[method_name]['Price']) > 0:
            df_true_values[f'{method_name}_Price'] = predictions_data[method_name]['Price']
            df_true_values[f'{method_name}_Demand'] = predictions_data[method_name]['Demand']
    
    # Save all DataFrames to CSV
    df_epoch_metrics.to_csv('epoch_metrics.csv', index=False)
    df_detailed.to_csv('detailed_results.csv', index=False)
    df_summary.to_csv('method_summary.csv', index=False)
    df_true_values.to_csv('predictions.csv', index=False)

    # Calculate forecasting errors (RMSE)
    forecasting_errors = calculate_forecasting_errors(df_true_values)
    
    print("\nResults Summary:")
    print(df_summary.to_string(index=False, float_format=lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A"))

    print("\nForecasting Errors (RMSE):")
    print(forecasting_errors.to_string(index=False, float_format=lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A"))


    print("\nAll results have been saved to CSV files:")
    print("- epoch_metrics.csv: Detailed metrics for each epoch")
    print("- detailed_results.csv: Results for each method and step")
    print("- method_summary.csv: Summary statistics for each method")
    print("- predictions.csv: True values and predictions for test windows")
    print("- forecasting_errors.csv: RMSE for price and demand predictions")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load data
df = pd.read_csv("epoch_metrics.csv")


# Extract training and test dataset sizes
train_sizes = df['Window_Size'].unique()
avg_train_size = df['Window_Size'].mean()
min_train_size = df['Window_Size'].min()
max_train_size = df['Window_Size'].max()


# Group by Method and Epoch to handle multiple steps
grouped_df = df.groupby(['Method', 'Epoch']).agg({
    'Train_Regret': 'mean',
    'Test_Regret': 'mean'
}).reset_index()

# Calculate average regret across all epochs for each method
avg_results = grouped_df.groupby('Method').agg({
    'Train_Regret': 'mean',
    'Test_Regret': 'mean'
}).reset_index()

# Sort the methods based on average regret across all epochs (lower is better)
train_method_order = avg_results.sort_values('Train_Regret')['Method'].tolist()
test_method_order = avg_results.sort_values('Test_Regret')['Method'].tolist()

# Get the final epoch values for annotation purposes
final_epoch = grouped_df['Epoch'].max()
final_results = grouped_df[grouped_df['Epoch'] == final_epoch].copy()

# Create pivot tables for heatmaps
pivot_train = grouped_df.pivot(index='Method', columns='Epoch', values='Train_Regret')
pivot_test = grouped_df.pivot(index='Method', columns='Epoch', values='Test_Regret')

# Reorder methods based on average performance - each with its own ordering
pivot_train = pivot_train.reindex(train_method_order)
pivot_test = pivot_test.reindex(test_method_order)

# Add ranking to method names - include only average performance
train_ranked_methods = [
    f"{i+1}. {method} (avg: {avg_results[avg_results['Method']==method]['Train_Regret'].values[0]:.3f})" 
    for i, method in enumerate(train_method_order)
]

test_ranked_methods = [
    f"{i+1}. {method} (avg: {avg_results[avg_results['Method']==method]['Test_Regret'].values[0]:.3f})" 
    for i, method in enumerate(test_method_order)
]

# Apply the rankings
pivot_train.index = train_ranked_methods
pivot_test.index = test_ranked_methods

# Create figure and subplots
fig, axes = plt.subplots(2, 1, figsize=(30, 14))

# Plot Training Regret Heatmap (sorted by average Train Regret)
sns.heatmap(
    pivot_train, 
    ax=axes[0], 
    cmap="Greens", 
    annot=True, 
    fmt=".3f",
    cbar_kws={'label': 'Regret (lower is better)'},
    linecolor='white',
    linewidths=0.05,
    annot_kws={"size": 14}
)
axes[0].patch.set_alpha(0.1)
# In walk-forward validation, initial training set = total windows - test_size
# Get initial window size (first step)
initial_train_windows = df[df['Step'] == 1]['Window_Size'].iloc[0]
initial_train_days = int(initial_train_windows / 1)  # Each window is 1 day

axes[0].set_title(f"Train Regret - Initial train set: {initial_train_days} days", fontsize=20)
axes[0].set_ylabel("Methods (ranked by average Train Regret)", fontsize=18)
axes[0].set_xlabel("Epoch", fontsize=18)

# For training heatmap
for label in axes[0].get_yticklabels():
    label.set_fontsize(14)
    label.set_fontweight('bold')
    label.set_rotation(0)

# Plot Test Regret Heatmap (sorted by average Test Regret)
sns.heatmap(
    pivot_test, 
    ax=axes[1], 
    cmap="Greens", 
    annot=True, 
    fmt=".3f",
    cbar_kws={'label': 'Regret (lower is better)'},
    linecolor='white',
    linewidths=0.05,
    annot_kws={"size": 14}
)
axes[1].patch.set_alpha(0.5)
axes[1].set_title(f"Test Regret: {test_size} days", fontsize=20)
axes[1].set_ylabel("Methods (ranked by average Test Regret)", fontsize=18)
axes[1].set_xlabel("Epoch", fontsize=18)

# For test heatmap
for label in axes[1].get_yticklabels():
    label.set_fontsize(14)
    label.set_fontweight('bold')
    label.set_rotation(0)

plt.suptitle("Method Comparison - Sorted by Average Regret Values Across All Epochs (lower is better)", fontsize=20)

# Add explanation of regret as a text box
explanation = (
    "Regret: The absolute difference between the objective value achieved when using predicted prices (with true demand constraints) versus the objective value achieved with true prices (and true demand constraints).\n"
    "Formula: regret = cost_with_predicted_prices - cost_with_true_prices\n"
    "• Lower values indicate decisions closer to optimal.\n"
    "• Higher values indicate more suboptimal decisions.\n"
    "• Methods are ranked by their average regret across all test set.\n"
    "(The cost in this battery scheduling problem is the minimisation objective, which is calculated as the sum of grid power usage multiplied by electricity prices at each time interval, plus the cost of battery charging.\n The optimisation model aims to minimise this total cost while satisfying demand and battery constraints.)"
)
plt.figtext(0.5, 0.05, explanation, ha="center", fontsize=14, 
            bbox={"facecolor":"white", "alpha":1, "pad":5, "edgecolor":"lightgray"})

plt.tight_layout(rect=[0, 0.15, 1, 0.97])  # Make room for suptitle and explanation
plt.savefig('regrets_with_sizes.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load data
df = pd.read_csv("epoch_metrics.csv")

# Print some diagnostic information about the data
print("Summary of regret values in epoch_metrics.csv:")
print(df.groupby('Method').agg({
    'Train_Regret': ['min', 'max', 'mean', 'median', 'count'],
    'Test_Regret': ['min', 'max', 'mean', 'median', 'count']
}))

# Find any extreme values in the Train_Regret column
extreme_values = df[df['Train_Regret'] > 10]
if not extreme_values.empty:
    print("\nExtreme Train_Regret values detected:")
    print(extreme_values[['Method', 'Step', 'Epoch', 'Train_Regret']])

# If needed, cap extreme values for visualization purposes
if df['Train_Regret'].max() > 10:
    print("\nCapping extreme Train_Regret values at 10.0 for visualization")
    df['Train_Regret'] = df['Train_Regret'].clip(upper=10.0)

# Extract training and test dataset sizes
train_sizes = df['Window_Size'].unique()
avg_train_size = df['Window_Size'].mean()
min_train_size = df['Window_Size'].min()
max_train_size = df['Window_Size'].max()

# Use Step instead of Epoch as the grouping factor
grouped_df = df.groupby(['Method', 'Step']).agg({
    'Train_Regret': 'mean',
    'Test_Regret': 'mean',
    'Window_Size': 'first'  # Keep window size information
}).reset_index()

# Calculate average regret across all steps for each method
avg_results = grouped_df.groupby('Method').agg({
    'Train_Regret': 'mean',
    'Test_Regret': 'mean'
}).reset_index()

print("\nAverage results across all steps:")
print(avg_results)

# Sort the methods based on average regret across all steps (lower is better)
train_method_order = avg_results.sort_values('Train_Regret')['Method'].tolist()
test_method_order = avg_results.sort_values('Test_Regret')['Method'].tolist()

# Create pivot tables for heatmaps
pivot_train = grouped_df.pivot(index='Method', columns='Step', values='Train_Regret')
pivot_test = grouped_df.pivot(index='Method', columns='Step', values='Test_Regret')

# Reorder methods based on average performance - each with its own ordering
pivot_train = pivot_train.reindex(train_method_order)
pivot_test = pivot_test.reindex(test_method_order)

# Add ranking to method names - include only average performance
train_ranked_methods = [
    f"{i+1}. {method} (avg: {avg_results[avg_results['Method']==method]['Train_Regret'].values[0]:.3f})" 
    for i, method in enumerate(train_method_order)
]

test_ranked_methods = [
    f"{i+1}. {method} (avg: {avg_results[avg_results['Method']==method]['Test_Regret'].values[0]:.3f})" 
    for i, method in enumerate(test_method_order)
]

# Apply the rankings
pivot_train.index = train_ranked_methods
pivot_test.index = test_ranked_methods

# Extract window sizes for each step to include in the x-axis labels
step_window_sizes = grouped_df.groupby('Step')['Window_Size'].first().to_dict()
x_labels = [f"{step} ({size} days)" for step, size in step_window_sizes.items()]

# Create figure and subplots
fig, axes = plt.subplots(2, 1, figsize=(18, 12))

# Plot Training Regret Heatmap (sorted by average Train Regret)
sns.heatmap(
    pivot_train, 
    ax=axes[0], 
    cmap="Greens", 
    annot=True, 
    fmt=".3f",
    cbar_kws={'label': 'Regret (lower is better)'},
    linecolor='white',
    linewidths=0.05,
    annot_kws={"size": 12}
)
axes[0].patch.set_alpha(0.1)

# Update x-axis labels to include window sizes
axes[0].set_xticklabels(x_labels, rotation=45, ha='right')

axes[0].set_title(f"Train Regret by Step (Window Size Increasing)", fontsize=20)
axes[0].set_ylabel("Methods (ranked by average Train Regret)", fontsize=18)
axes[0].set_xlabel("Step (Window Size in days)", fontsize=18)

# For training heatmap
for label in axes[0].get_yticklabels():
    label.set_fontsize(14)
    label.set_fontweight('bold')
    label.set_rotation(0)

# Plot Test Regret Heatmap (sorted by average Test Regret)
sns.heatmap(
    pivot_test, 
    ax=axes[1], 
    cmap="Greens", 
    annot=True, 
    fmt=".3f",
    cbar_kws={'label': 'Regret (lower is better)'},
    linecolor='white',
    linewidths=0.05,
    annot_kws={"size": 12}
)
axes[1].patch.set_alpha(0.5)

# Update x-axis labels to include window sizes
axes[1].set_xticklabels(x_labels, rotation=45, ha='right')

# Extract test size from the data if available, otherwise use a default value
test_size = 14  # Default value
if 'test_size' in globals():
    pass  # Use the value from globals
elif hasattr(df, 'test_size'):
    test_size = df.test_size
    
axes[1].set_title(f"Test Regret by Step - Test Size: {test_size} days", fontsize=20)
axes[1].set_ylabel("Methods (ranked by average Test Regret)", fontsize=18)
axes[1].set_xlabel("Step (Window Size in days)", fontsize=18)

# For test heatmap
for label in axes[1].get_yticklabels():
    label.set_fontsize(14)
    label.set_fontweight('bold')
    label.set_rotation(0)

plt.suptitle("Method Comparison by Step - Sorted by Average Regret Values (lower is better)", fontsize=20)

# Add explanation of regret as a text box
explanation = (
    "Regret: The absolute difference between the objective value achieved when using predicted prices (with true demand constraints) versus the objective value achieved with true prices (and true demand constraints).\n"
    "Formula: regret = cost_with_predicted_prices - cost_with_true_prices\n"
    "• Lower values indicate decisions closer to optimal.\n"
    "• Higher values indicate more suboptimal decisions.\n"
    "• Methods are ranked by their average regret across all test set.\n"
    "(The cost in this battery scheduling problem is the minimisation objective, which is calculated as the sum of grid power usage multiplied by electricity prices at each time interval, plus the cost of battery charging.\n The optimisation model aims to minimise this total cost while satisfying demand and battery constraints.)"
)
plt.figtext(0.5, 0.05, explanation, ha="center", fontsize=14, 
            bbox={"facecolor":"white", "alpha":1, "pad":5, "edgecolor":"lightgray"})

plt.tight_layout(rect=[0, 0.15, 1, 0.97])  # Make room for suptitle and explanation
plt.savefig('regrets_by_step.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load data
df = pd.read_csv("epoch_metrics.csv")

# Group by Method and Epoch to handle multiple steps
grouped_df = df.groupby(['Method', 'Epoch']).agg({
    'Epoch_Time': 'mean'  # Using Epoch_Time instead of regret
}).reset_index()

# Get the average time across all epochs for each method
avg_times = grouped_df.groupby('Method')['Epoch_Time'].mean().reset_index()

# Sort the methods by average time (lower is better/faster)
method_order = avg_times.sort_values('Epoch_Time')['Method'].tolist()

# Create pivot tables for heatmaps with sorted methods
pivot_time = grouped_df.pivot(index='Method', columns='Epoch', values='Epoch_Time')

# Reorder methods based on performance
pivot_time = pivot_time.reindex(method_order)

# Add ranking to method names
ranked_methods = [
    f"{i+1}. {method} ({avg_times[avg_times['Method'] == method]['Epoch_Time'].values[0]:.4f}s)"
    for i, method in enumerate(method_order)
]
pivot_time.index = ranked_methods

# Create figure and subplots
fig, ax = plt.subplots(figsize=(14, 8))

# Use a different colormap for time (lower is better)
sns.heatmap(
    pivot_time, 
    ax=ax, 
    cmap="Greens",  
    annot=True, 
    fmt=".4f",
    cbar_kws={'label': 'Processing Time (seconds)'}
)

# Rotate y-axis labels vertically for readability
for label in ax.get_yticklabels():
    label.set_fontsize(12)
    label.set_fontweight('bold')
    label.set_rotation(0)

ax.set_title("Processing Time per Epoch", fontsize=16)
ax.set_ylabel("Methods (ranked by average processing time)", fontsize=14)
ax.set_xlabel("Epoch", fontsize=14)

plt.suptitle("Method Comparison - Sorted by Average Processing Time (Lower is Faster)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('Processing_Time.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('forecasting_errors.csv')
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))

for ax, col in zip([ax1, ax2], ['Price_RMSE', 'Demand_RMSE']):
    bars = ax.bar(df['Method'], df[col], zorder=2, fill=True, color='lightsteelblue', alpha=0.5, edgecolor='black', linewidth=1.2)
    ax.set_title(col.replace('_', ' '))
    ax.set_ylabel('RMSE - Test Set')
    ax.grid(axis='y', linestyle='--', alpha=0.7, zorder=1)
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
    ax.yaxis.set_major_locator(plt.MaxNLocator(nbins=15))
    for bar in bars:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{bar.get_height():.2f}',
                ha='center', va='bottom', fontsize=8)

for ax in [ax1, ax2]:
    ax.tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.savefig('RMSE.png', dpi=300, bbox_inches='tight')
plt.show()
